# Лабораторная работа 2. Метод ближайших соседей и решающие деревья.

ФИО:  Алескин Александр Сергеевич

Группа: 317

In [1]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [5]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [6]:
data.shape

(32769, 10)

In [7]:
# доля положительных примеров
data.ACTION.mean()

0.9421099209618847

In [8]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

SyntaxError: invalid syntax (<ipython-input-8-99e87b3b8f79>, line 3)

In [9]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

In [10]:
print y_train.mean()
print y_test.mean()

SyntaxError: invalid syntax (<ipython-input-10-89c72e1c0b22>, line 1)

## Часть 1: kNN и категориальные признаки

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре. Реализуйте самостоятельно метод k ближайших соседей, который будет уметь работать с этими функциями расстояния. Подсчитайте для каждой из них качество на тестовой выборке `X_test` при числе соседей $k = 10$. Метрика качества — AUC-ROC.

Какая функция расстояния оказалась лучшей?

In [8]:
# metrics

def match_id(data, obj, p, p2):
    """
    p and p2 for beauty in kNN
    return vector of distances
    
    """
    return (data != obj).sum(axis = 1)

def smooth_match_id(data, obj, data_w, p_val):
    """
    p_val - probability of that 2 object have same value
    return vector of distances
    
    """
    non_equal = (data != obj)
    match = np.sum(non_equal, axis = 1)
    dist =  (data == obj) * data_w
    dist = np.sum(dist, axis = 1)
    return match + dist

def match_log_id(data, obj, data_log, p_val_log):
    """
    p_val_log - log of frequency of value
    return vector of distances
    
    """
    p_id = data != obj
    dist = np.zeros((p_id.shape[0]))
    for i in np.arange(p_id.shape[1]):
        arr = p_val_log[i]
        try:
            obj_log = arr[obj[i]]
            dist += p_id[:,i] * data_log[:,i] * obj_log
        except KeyError:
            pass
        
    return dist
        


In [9]:
# kNN    
    
def neighbor_class(metrik_func, k, neighbours, ans_neigs, data_to_class):
    """
    k - amount_neig
    
    metrik_func:
    mi - match id
    smi - smooth_match_id
    mli - match_log_id
    
    """
    neigs = neighbours.values
    data = data_to_class.values
    #neigs_ans = ans_neigs.values 
    weights = []
    data_w = []
    if metrik_func =='smi':
        metric = smooth_match_id
        size_n = neigs.shape[0]
        data_w = np.zeros(neigs.shape)
        for i in range(neigs.shape[1]):
            x_val, x_count = np.unique(neigs[:, i] , return_counts = True)
            p_vals = pd.Series(np.float32(x_count * (x_count - 1))/(size_n * (size_n - 1)), index = x_val)
            p_vals.sort()
            data_w[:, i] = [np.sum(p_vals[0:np.where(p_vals.index == j)[0][0]+1]) 
                            for j in neigs[:, i]]
            
    elif metrik_func == 'mli':
        metric = match_log_id
        size_n = neigs.shape[0]
        data_w = np.empty_like(neigs)
        for i in range(neigs.shape[1]):
            x_val, x_count = np.unique(neigs[:, i] , return_counts = True)
            p_vals = pd.Series(np.log(x_count), index = x_val)
            data_w[:,i] = p_vals[neigs[:,i]]
            weights += [p_vals]
        
    elif metrik_func == 'mi':
        metric = match_id
    else:
        raise ValueError ("no", metrik_func)
    
    ans_data = np.empty(data.shape[0])
    for i in range(data.shape[0]):
        dist = metric(neigs, data[i], data_w, weights)
        pd_dist = pd.Series(dist, index = neighbours.index)
        pd_dist.sort()
        ans_data[i] = float(np.sum(ans_neigs[pd_dist[0:k].index])) / k
    return pd.Series(ans_data, data_to_class.index)
        

In [10]:
n_neighbours = 10

y_mi = neighbor_class('mi', n_neighbours,  X_train, y_train, X_test)

In [11]:

y_smi = neighbor_class('smi', n_neighbours,  X_train, y_train, X_test)

In [12]:

y_mli = neighbor_class('mli', n_neighbours,  X_train, y_train, X_test)

In [13]:
from sklearn.metrics import roc_auc_score
print "match_id gives ", roc_auc_score(y_test, y_mi)
print "smooth_match_id gives ", roc_auc_score(y_test, y_smi)
print "match_log_id ", roc_auc_score(y_test, y_mli)

match_id gives  0.83088009598
smooth_match_id gives  0.832857727689
match_log_id  0.802592208011


######  МОИ Комментарии: вообще столько значений у некоторых параметров, что не все значения попадают в обучающую выборку.  Наилучший результат у сглаживающего признака

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось достичь?

In [14]:
k_neig = np.arange(2, 70, 5)
mi_mass = np.zeros_like(k_neig, dtype = float)
smi_mass = np.zeros_like(k_neig, dtype = float)
mli_mass = np.zeros_like(k_neig, dtype = float)
for i in range(k_neig.size):
    mi_mass[i] = roc_auc_score(y_test, neighbor_class('mi', k_neig[i],  X_train, y_train, X_test))
    smi_mass[i] = roc_auc_score(y_test, neighbor_class('smi', k_neig[i],  X_train, y_train, X_test))
    mli_mass[i] = roc_auc_score(y_test, neighbor_class('mli', k_neig[i],  X_train, y_train, X_test))
print "best score mi  ", np.amax(mi_mass), ' with k = ', k_neig[np.argmax(mi_mass)]
print "best score smi ", np.amax(smi_mass), ' with k = ', k_neig[np.argmax(smi_mass)]
print "best score mli ", np.amax(mli_mass), ' with k = ', k_neig[np.argmax(mli_mass)]

best score mi   0.829705773258  with k =  12
best score smi  0.836041490999  with k =  7
best score mli  0.807762498733  with k =  12


In [16]:
k_neig = np.arange(3, 15, 1)
mi_mass = np.zeros_like(k_neig, dtype = float)
smi_mass = np.zeros_like(k_neig, dtype = float)
mli_mass = np.zeros_like(k_neig, dtype = float)
for i in range(k_neig.size):
    mi_mass[i] = roc_auc_score(y_test, neighbor_class('mi', k_neig[i],  X_train, y_train, X_test))
    smi_mass[i] = roc_auc_score(y_test, neighbor_class('smi', k_neig[i],  X_train, y_train, X_test))
    mli_mass[i] = roc_auc_score(y_test, neighbor_class('mli', k_neig[i],  X_train, y_train, X_test))
print "best score mi  ", np.amax(mi_mass), ' with k = ', k_neig[np.argmax(mi_mass)]
print "best score smi ", np.amax(smi_mass), ' with k = ', k_neig[np.argmax(smi_mass)]
print "best score mli ", np.amax(mli_mass), ' with k = ', k_neig[np.argmax(mli_mass)]

best score mi   0.831201958708  with k =  11
best score smi  0.836041490999  with k =  7
best score mli  0.807762498733  with k =  12


### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `clicks` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`clicks` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `clicks` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [4]:
# мои функции для счетчиков и фолдинга

import sklearn as sk 

def item_counts(vals, ans, a, b):
    """
    return counts of feature
    
    """
    x_val, x_count = np.unique(vals , return_counts = True)
    counts = x_count
    clicks = np.array([np.sum(ans[np.where(vals == j)[0]] == 1) for j in x_val])
    counts = np.float16(counts) / vals.size
    clicks = np.float16(clicks) / vals.size
    p = (clicks + a) / (counts + b)
    return_val = np.reshape(np.concatenate((counts, clicks, p)), (p.size, -1), 'F')
    return pd.DataFrame(data = return_val, index = x_val)

def counters(data_test, data_train, answers, a = 1, b = 2):
    ans = answers.values
    vals = data_train.values
    test = data_test.values
    props = data_train.shape[1] # features in data
    data2 = pd.DataFrame(data = np.empty((test.shape[0], props * 3), dtype = int), index = data_test.index)
    for i in range(props):
        item_counters = item_counts(vals[:, i], ans, a, b)
        uniq = np.unique([j for j in test[:, i] if not (vals[:,i] == j).any()])
        extra_data = np.zeros((uniq.size, 3))
        extra_frame = pd.DataFrame(data = extra_data, index = uniq)
        it_cs = item_counters.append(extra_frame)
        data2.iloc[:, i * 3 : (i + 1) * 3] = it_cs.ix[test[:, i]].values
    return data2 

def folding_counters(x_test, x_train, y_train, n = 3, a = 1, b = 2):
    
    x_test2 = counters(x_test, x_train, y_train, a, b) # make x_test
    
    tr_sh = x_train.shape
    x_train2 = pd.DataFrame(data = np.empty((tr_sh[0], tr_sh[1] * 3)), index = x_train.index)
    kf = sk.cross_validation.KFold(x_train.shape[0], n_folds = n, shuffle = True)
    for train_index, test_index in kf:
        x_train2.iloc[test_index, :] = counters(x_train.iloc[test_index, :], x_train.iloc[train_index, :],
                                           y_train.iloc[train_index], a, b)
    return x_test2, x_train2
        


#### без фолдинга

In [14]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score

In [19]:
X_test2 = counters(X_test, X_train, y_train)
X_train2 = counters(X_train, X_train, y_train)
n = np.arange(2, 40, 2)
auc = np.zeros(n.shape)
for j in range(n.size):
    knn = NearestNeighbors(n_neighbors = n[j], metric = 'euclidean').fit(X_train2)
    neighs = knn.kneighbors(X_test2, return_distance = False)
    y_prec = np.zeros(y_test.shape)
    for i in range(j):
        y_prec += y_train.iloc[neighs[:, i]]
    y_prec /= n[j]
    auc[j] = roc_auc_score(y_test, y_prec)

In [20]:
print "best score  ", np.amax(auc), ' with n = ', n[np.argmax(auc)]

best score   0.796083338545  with n =  30


#### c фолдингом

In [15]:
X_test2, X_train2 = folding_counters(X_test, X_train, y_train, n = 5)
n = np.arange(2, 100, 4)
auc = np.zeros(n.shape)
for j in range(n.size):
    knn = NearestNeighbors(n_neighbors = n[j], metric = 'euclidean').fit(X_train2)
    neighs = knn.kneighbors(X_test2, return_distance = False)
    y_prec = np.zeros(y_test.shape)
    for i in range(j):
        y_prec += y_train.iloc[neighs[:, i]]
    y_prec /= n[j]
    auc[j] = roc_auc_score(y_test, y_prec)

In [16]:
print "best score  ", np.amax(auc), ' with n = ', n[np.argmax(auc)]

best score   0.745712091184  with n =  74


#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $f_i$, $f_j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$. Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

In [23]:
data.max() # воспользуемся, что признаки ограничены. например миллионом 

ACTION                   1
RESOURCE            312153
MGR_ID              311696
ROLE_ROLLUP_1       311178
ROLE_ROLLUP_2       286791
ROLE_DEPTNAME       286792
ROLE_TITLE          311867
ROLE_FAMILY_DESC    311867
ROLE_FAMILY         308574
ROLE_CODE           270691
dtype: int64

In [24]:
# добавление признаков
data2 = data.copy()
for i in range(1, data.shape[1]):
    for j in range(i+1, data.shape[1]):
        data2[str(i) + '+' + str(j)] = data.iloc[:, i]*(10**6) + data.iloc[:, j]

X_train2, X_test2, y_train2, y_test2 = train_test_split(data2.iloc[:, 1:], data2.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
X_train2.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,1+2,...,5+6,5+7,5+8,5+9,6+7,6+8,6+9,7+8,7+9,8+9
1728,40867,8150,117961,118386,119954,118054,118054,117887,118055,40867008150,...,119954118054,119954118054,119954117887,119954118055,118054118054,118054117887,118054118055,118054117887,118054118055,117887118055
22856,38587,14551,5110,117954,117895,117899,117897,19721,117900,38587014551,...,117895117899,117895117897,117895019721,117895117900,117899117897,117899019721,117899117900,117897019721,117897117900,19721117900
27691,3853,51301,118290,118291,121108,126869,118302,118295,126870,3853051301,...,121108126869,121108118302,121108118295,121108126870,126869118302,126869118295,126869126870,118302118295,118302126870,118295126870
8421,45337,49423,117961,118052,123144,118396,269406,118398,118399,45337049423,...,123144118396,123144269406,123144118398,123144118399,118396269406,118396118398,118396118399,269406118398,269406118399,118398118399
5013,40919,87700,118315,118463,118464,118465,162182,118467,118468,40919087700,...,118464118465,118464162182,118464118467,118464118468,118465162182,118465118467,118465118468,162182118467,162182118468,118467118468


#### без фолдинга

In [25]:
X_test3 = counters(X_test2, X_train2, y_train)
X_train3 = counters(X_train2, X_train2, y_train)
n = np.arange(2, 100, 4)
auc = np.zeros(n.shape)
for j in range(n.size):
    knn = NearestNeighbors(n_neighbors = n[j], metric = 'euclidean').fit(X_train3)
    neighs = knn.kneighbors(X_test3, return_distance = False)
    y_prec = np.zeros(y_test.shape)
    for i in range(j):
        y_prec += y_train.iloc[neighs[:, i]]
    y_prec /= n[j]
    auc[j] = roc_auc_score(y_test, y_prec)

In [26]:
print "best score  ", np.amax(auc), ' with n = ', n[np.argmax(auc)]

best score   0.801776319242  with n =  34


#### c фолдингом (и по нормальному)

In [27]:
from sklearn.neighbors import KNeighborsClassifier
X_test3, X_train3 = folding_counters(X_test2, X_train2, y_train2, n = 5)

In [30]:
n = np.arange(2,100, 4)
auc = np.zeros(n.shape)
for j in range(n.size):
    knn = KNeighborsClassifier(n_neighbors = n[j]).fit(X_train3, y_train)
    y_prec = knn.predict_proba(X_test3)
    auc[j] = roc_auc_score(y_test, y_prec[:, 1])


In [31]:
print "best score  ", np.amax(auc), ' with n = ', n[np.argmax(auc)]

best score   0.666292687696  with n =  18


## Часть 2: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

In [32]:
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [33]:
X_test3 = counters(X_test2, X_train2, y_train)
X_train3 = counters(X_train2, X_train2, y_train)

In [34]:
leaves = np.arange(2, 50, 3)
deep  = np.arange(4, 50, 4)
gs = GridSearchCV(DecisionTreeClassifier(random_state = 241),
                  param_grid = {"max_depth": deep, 
                                'min_samples_leaf': leaves})
gs.fit(X_train3, y_train)



GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=241, splitter='best'),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'max_depth': array([ 4,  8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48]), 'min_samples_leaf': array([ 2,  5,  8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [35]:
prec_y = gs.predict_proba(X_test3)
print " best score is ", roc_auc_score(y_test, prec_y[:, 1])
print " best parameters are", gs.best_params_

 best score is  0.534879155012
 best parameters are {'max_depth': 4, 'min_samples_leaf': 2}


#### 2. Настройте случайный лес, подобрав оптимальное число деревьев `n_estimators`. Какое качество на тестовой выборке он дает?

In [36]:
n = np.arange(1, 40)**2
gsf = GridSearchCV(RandomForestClassifier(random_state = 241),
                  param_grid = {'n_estimators': n})
gsf.fit(X_train3, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=241, verbose=0, warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_estimators': array([   1,    4,    9,   16,   25,   36,   49,   64,   81,  100,  121,
        144,  169,  196,  225,  256,  289,  324,  361,  400,  441,  484,
        529,  576,  625,  676,  729,  784,  841,  900,  961, 1024, 1089,
       1156, 1225, 1296, 1369, 1444, 1521])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [37]:
prec_y = gsf.predict_proba(X_test3)
print " best score is ", roc_auc_score(y_test, prec_y[:, 1])
print " best parameters are", gsf.best_params_

 best score is  0.643255601058
 best parameters are {'n_estimators': 25}


#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

In [38]:
X_test4, X_train4 = folding_counters(X_test2, X_train2, y_train2, n = 5)


In [39]:
n = np.arange(1, 40)**2
gsf2 = GridSearchCV(RandomForestClassifier(random_state = 241),
                  param_grid = {'n_estimators': n})
gsf2.fit(X_train4, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=241, verbose=0, warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_estimators': array([   1,    4,    9,   16,   25,   36,   49,   64,   81,  100,  121,
        144,  169,  196,  225,  256,  289,  324,  361,  400,  441,  484,
        529,  576,  625,  676,  729,  784,  841,  900,  961, 1024, 1089,
       1156, 1225, 1296, 1369, 1444, 1521])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [40]:
prec_y = gsf2.predict_proba(X_test4) 
print " best score is ", roc_auc_score(y_test, prec_y[:, 1])
print " best parameters are", gsf2.best_params_

 best score is  0.845526377648
 best parameters are {'n_estimators': 324}


Вероятной причиной является, как было отмечено в условии пункта 1.3, "признаки, содержащие информацию о целевой переменной, могут привести к переобучению". Большое количество значений признаков и сравнительное небольшое размер выборки увеличивает количество объектов, которые имеют уникальное значение признака(ов), особенно после добавления парных признаков. Причем clicks для уникальных признаков у таких объектов будет тождественно равен целевой переменной, что прямо отражается на переобучаемости дерева(леса деревьев). 